In [61]:
import polars as pl 
import numpy as np
cov = 0.5


elongates = pl.read_csv(f"output/{cov}/{cov}_elongates.csv", infer_schema_length =10000)

def gini_coefficient(x):
    
    """
    Compute Gini coefficient of array of values
    Source : https://stackoverflow.com/questions/39512260/calculating-gini-coefficient-in-python-numpy by Ulf Aslak
    """

    diffsum = 0
    for i, xi in enumerate(x[:-1], 1):
        
        print(i,xi)
        diffsum += np.sum(np.abs(xi - x[i:]))
        print(diffsum)
    return diffsum / (len(x)**2 * np.mean(x))

In [76]:
from utils.handle_UTRs import translate_frames
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import numpy as np

def custom_target_elongate(cluster, scer_length, seq_id, specie, elongate_length, side, genome_dict, gff_dict, coeff = 1): 


    coordinates = list()
    result_dict = dict()

    if side != "Nter" and side != "Cter":

        raise ValueError("Side must be either Nter or Cter")
    
    gff = gff_dict[specie].filter(

        (pl.col("Type") == "CDS") & ((pl.col("Name") == seq_id) | (pl.col("Parent") == seq_id))

    )[["Start","End","Strand","Seqid"]] # Keep only necessary columns
    
    # Store datas necessary to compute the elongate sequence

    strand = gff[0]["Strand"].to_list()[0] # + or -
    strand_id = gff[0]["Seqid"].to_list()[0] # chromosome or scaffold id

    for row in gff.iter_rows(named=True): # Named = True to iter with column names

        coordinates.append(sorted((int(row['Start'])-1, int(row['End'])-1))) # -1 for python indexing
        
    coordinates = sorted(coordinates, key=lambda x: x[0]) # Sort coordinates by start position


    custom_elongate_length = int(np.ceil((scer_length - elongate_length) * 3 * coeff)) # Get the length of the elongate sequence in nucleotides

    if strand == "+":

        if side == "Nter":

            start_5 = coordinates[0][0]-custom_elongate_length if coordinates[0][0]-custom_elongate_length >= 0 else 0 # Get the start position of the 5' UTR
            
            elongate = genome_dict[specie][strand_id]["seq"][

                start_5:coordinates[0][0]

            ] 

        elif side == "Cter": # Useless check but it's for the sake of clarity
        
            end_3 = coordinates[-1][1]+1+custom_elongate_length if coordinates[-1][1]+1+custom_elongate_length <= genome_dict[specie][strand_id]["len"] else genome_dict[specie][strand_id]["len"] # Get the end position of the 3' UTR
            # +1 for -1,1 because GFF points to the last nucleotide of the stop codon

            elongate = genome_dict[specie][strand_id]["seq"][
                coordinates[-1][1]+1:end_3
                ] # Get the 3' sequence

    
    # Reverse complement if the strand is negative, don't forget to reverse the coordinates
    if strand == "-":

        if side == "Nter":

            end_5 = coordinates[-1][1]+1+custom_elongate_length if coordinates[-1][1]+1+custom_elongate_length <= genome_dict[specie][strand_id]["len"] else genome_dict[specie][strand_id]["len"] # Get the start position of the 5' UTR

            elongate = genome_dict[specie][strand_id]["seq"][
                coordinates[-1][1]+1:end_5
            ].reverse_complement() # Get the 5' sequence
        

        if side == "Cter":
        
            start_3 = coordinates[0][0]-custom_elongate_length if coordinates[0][0]-custom_elongate_length >= 0 else 0 # Get the end position of the 3' UTR

            elongate = genome_dict[specie][strand_id]["seq"][
                start_3:coordinates[0][0]
            ].reverse_complement() # Get the 3' sequence


    result_dict["nuc"] = SeqRecord(seq = Seq(elongate), id = f"{seq_id}-{cluster}", description = "")

    # def translate_frames(dna_sequence, specie, seq_id, length, utr, cluster)

    result_dict["prot"] = translate_frames(dna_sequence = elongate, specie = specie, seq_id = seq_id, length = custom_elongate_length, utr = side, cluster = cluster)

    return result_dict


In [62]:
import yaml 

with open('/home/sherman/Bureau/Gits/Elongates/env.yaml', 'r') as f:
    yaml_data = yaml.safe_load(f)
    species_order = yaml_data['Species_order']['Scer']

In [71]:
seuil = 4

Nter_scer_conditions = ((pl.col("species") == "Scer_NCBI") & (abs(pl.col("max_Nter") - pl.col("Nter_nb_aa")) < seuil) & (pl.col("max_Nter") >= 10))
Cter_scer_conditions = ((pl.col("species") == "Scer_NCBI") & (abs(pl.col("max_Cter") - pl.col("Cter_nb_aa")) < seuil) & (pl.col("max_Cter") >= 10))
test = elongates.filter(Nter_scer_conditions | Cter_scer_conditions)["cluster_name"].unique().to_list()

nter_clusters = []
cter_clusters = []
i = 0

for cluster,sequences in elongates.filter(pl.col("cluster_name").is_in(test)).groupby("cluster_name"):

    bool_ = True
    species = sequences["species"].to_list()

    species_ordered = [ s for s in species_order if s in species]
    
    if species_ordered[-1] != "Scer_NCBI" and species.count(species_ordered[-1]) > 1:
            
        bool_ = False
        

    if bool_ :

        if sequences.filter(pl.col("species") == species_ordered[-1])["Nter_nb_aa"].to_list()[0] < seuil and sequences["max_Nter"].max() >= 10:
        
            nter_clusters.append(cluster)

        if sequences.filter(pl.col("species") == species_ordered[-1])["Cter_nb_aa"].to_list()[0] < seuil and sequences["max_Cter"].max() >= 10:
        
            cter_clusters.append(cluster)



full_filtered_Nter = elongates.filter(pl.col("cluster_name").is_in(nter_clusters))
full_filtered_Cter = elongates.filter(pl.col("cluster_name").is_in(cter_clusters))

    
full_filtered_Nter.write_csv(f"output/{cov}/{cov}_elongates_Nter.csv")
full_filtered_Cter.write_csv(f"output/{cov}/{cov}_elongates_Cter.csv")


In [77]:
import gff3_parser
from utils.files import multifasta_to_dict
import os

current_path = "/home/sherman/Bureau/Gits/Elongates/work"

dataframes = {
    "Nter": full_filtered_Nter,
    "Cter": full_filtered_Cter,
}

species = yaml.safe_load(open('env.yaml'))["Species_order"]["Scer"] # Species will be passed as argument in the future
elongates = pl.read_csv(f"output/{cov}/{cov}_elongates.csv", has_header = True, infer_schema_length = 5000)

gff_dict = dict()
genome_dict = dict()

for specie in species:

    gff_dict[specie] = pl.from_pandas(gff3_parser.parse_gff3(f"input/{specie}.gff", parse_attributes = True, verbose = False))
    genome_dict[specie] = multifasta_to_dict(f"input/{specie}.fna", genome = True)


for side in ["Nter"]:

    df = dataframes[side]

    #os.mkdir(f"{current_path}/local_align_files/{side}")

    for cluster, sequences in df.groupby("cluster_name"):

        #os.mkdir(f"{current_path}/local_align_files/{side}/{cluster}")

        scer_length = sequences.filter(pl.col("species") == "Scer_NCBI")[f"{side}_nb_aa"].max() # Maybe several Scer sequences in the cluster, we take the longest elongate

        infos_for_scer_sequences = dict()

        for sequence in sequences.iter_rows(named = True): 

            if scer_length - sequence[f"{side}_nb_aa"] >= 10:

                dict_ = custom_target_elongate(cluster, scer_length, sequence["seq_id"], sequence["species"], sequence[f"{side}_nb_aa"], side, genome_dict, gff_dict)

                infos_for_scer_sequences[sequence["seq_id"]] = sequence["Nter_nb_aa"] 


                #os.mkdir(f"{current_path}/local_align_files/{side}/{cluster}/{sequence['seq_id']}")
                #os.mkdir(f"{current_path}/local_align_files/{side}/{cluster}/{sequence['seq_id']}/nucleotide")
                #os.mkdir(f"{current_path}/local_align_files/{side}/{cluster}/{sequence['seq_id']}/protein")
        print(sequences[["cluster_name","seq_id","Nter_nb_aa","Cter_nb_aa"]])
        print(infos_for_scer_sequences)
        print(dict_)

shape: (5, 4)
┌──────────────┬────────────────────┬────────────┬────────────┐
│ cluster_name ┆ seq_id             ┆ Nter_nb_aa ┆ Cter_nb_aa │
│ ---          ┆ ---                ┆ ---        ┆ ---        │
│ str          ┆ str                ┆ i64        ┆ i64        │
╞══════════════╪════════════════════╪════════════╪════════════╡
│ cluster_n500 ┆ Sbay_4.400         ┆ 0          ┆ 0          │
│ cluster_n500 ┆ rna-XM_033908851.1 ┆ 11         ┆ 0          │
│ cluster_n500 ┆ rna-NM_001178498.1 ┆ 13         ┆ 0          │
│ cluster_n500 ┆ Sarb_02G02400.1    ┆ 11         ┆ 0          │
│ cluster_n500 ┆ Skud_2.277         ┆ 0          ┆ 0          │
└──────────────┴────────────────────┴────────────┴────────────┘
{'Sbay_4.400': 0, 'Skud_2.277': 0}
{'nuc': SeqRecord(seq=Seq('ATGGACACGGACTCAGGTTTTACAAGTGGTCATGAAACT'), id='Skud_2.277-cluster_n500', name='<unknown name>', description='', dbxrefs=[]), 'prot': None}
shape: (3, 4)
┌───────────────┬────────────────────┬────────────┬────────────┐
│ 